In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from sklearn.model_selection import train_test_split
from os.path import join as opj

train = pd.read_json("../data/train.json")
test = pd.read_json("../data/test.json")

In [3]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], 
                          X_band_2[:, :, :, np.newaxis],
                          ((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [4]:
#incident angle:
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
X_train_inc = np.array(train.inc_angle)
X_test_inc = np.array(test.inc_angle)

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model,load_model
from keras import initializers
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import layers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils import plot_model
from keras import regularizers

Using TensorFlow backend.
/Users/hanfeimao/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [12]:
def getModel(input_shape = (75,75,3), KernelSize = (5,5), Momentum = 0.99):
    
    X_input = Input(input_shape)
    #input_CNN = ZeroPadding2D((0, 0))(X_input)
    input_CNN = BatchNormalization(momentum = Momentum)(X_input)
    
    ## Input Layer
    input_CNN = Conv2D(32,kernel_size=KernelSize,padding='same', name='c11')(input_CNN)
    input_CNN = BatchNormalization(momentum = Momentum, name='b11')(input_CNN)
    input_CNN = Activation('elu')(input_CNN)
    input_CNN = MaxPooling2D((2,2),strides=(2, 2),name='m11')(input_CNN)
    #input_CNN = Dropout(0.25)(input_CNN)
    input_CNN = Conv2D(64,kernel_size=KernelSize,padding='same',name='c12')(input_CNN)
    input_CNN = BatchNormalization(momentum = Momentum,name='b12')(input_CNN)
    input_CNN = Activation('elu')(input_CNN)
    input_CNN = MaxPooling2D((2,2),strides=(2, 2),name='m12')(input_CNN)
    #input_CNN = Dropout(0.25)(input_CNN)
    
    ## First Residual
    input_CNN_residual = BatchNormalization(momentum=Momentum)(input_CNN)
    input_CNN_residual = Conv2D(128,kernel_size=KernelSize,padding='same')(input_CNN_residual)
    input_CNN_residual = BatchNormalization(momentum = Momentum)(input_CNN_residual)
    input_CNN_residual = Activation('elu')(input_CNN_residual)
    input_CNN_residual = Dropout(0.25)(input_CNN_residual)
    input_CNN_residual = Conv2D(64,kernel_size=KernelSize,padding='same')(input_CNN_residual)
    input_CNN_residual = BatchNormalization(momentum = Momentum)(input_CNN_residual)
    input_CNN_residual = Activation('elu')(input_CNN_residual)
    input_CNN_residual = Dropout(0.25)(input_CNN_residual)
    
    input_CNN_residual = Add()([input_CNN_residual,input_CNN])
    
    ## Top CNN
    top_CNN = Conv2D(128, kernel_size = KernelSize, padding ='same')(input_CNN_residual)
    top_CNN = BatchNormalization(momentum=Momentum)(top_CNN)
    top_CNN = Activation('elu')(top_CNN)
    top_CNN = MaxPooling2D((2,2),strides=(2, 2))(top_CNN)
    top_CNN = Conv2D(256, kernel_size = KernelSize, padding ='same')(top_CNN)
    top_CNN = BatchNormalization(momentum=Momentum)(top_CNN)
    top_CNN = Activation('elu')(top_CNN)
    top_CNN = Dropout(0.25)(top_CNN)
    top_CNN = MaxPooling2D((2,2),strides=(2, 2))(top_CNN)
    top_CNN = Conv2D(512, kernel_size = KernelSize, padding ='same')(top_CNN)
    top_CNN = BatchNormalization(momentum=Momentum)(top_CNN)
    top_CNN = Activation('elu')(top_CNN)
    top_CNN = Dropout(0.25)(top_CNN)
    top_CNN = MaxPooling2D((2,2),strides=(2, 2))(top_CNN)
    top_CNN = GlobalMaxPooling2D()(top_CNN)

    #Dense Layers
    #X = Flatten()(top_CNN)
    X = Dense(512)(top_CNN)
    X = BatchNormalization(momentum=Momentum)(X)
    X = Activation('elu')(X)
    X = Dropout(0.25)(X)
    X = Dense(256)(X)
    X = BatchNormalization(momentum=Momentum)(X)
    X = Activation('elu')(X)
    X = Dropout(0.25)(X)    
    X = Dense(1, activation='sigmoid')(X)
    
    model = Model(inputs = X_input, outputs = X, name='simple_resnet')
    return model



In [8]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

file_path = "../weights_resnet_with_dropout.hdf5"


In [9]:
#train_test_split
target_train=train['is_iceberg']
X_train_new = X_train/100+0.5
X_train_cv, X_valid, X_angle_train, X_angle_valid, y_train_cv, y_valid = train_test_split(X_train_new, 
                                        X_train_inc, target_train, random_state=6, train_size=0.75)


/Users/hanfeimao/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
#Data Augmentation
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
            #rescale=1./50,
            rotation_range=20,  horizontal_flip=True,  vertical_flip=True,
            width_shift_range = 0.30,  height_shift_range = 0.30,  zoom_range = 0.1)

test_datagen = ImageDataGenerator(
            #rescale=1./50, 
            horizontal_flip=True, vertical_flip=True)

train_generator = train_datagen.flow(X_train_cv, y_train_cv, batch_size=batch_size)  
validation_generator = test_datagen.flow(X_valid, y_valid, batch_size=batch_size)


In [13]:
model = getModel()
# Optimizer
myoptim=Adam(lr=0.0004, beta_1=0.9, beta_2=0.999, epsilon=1e-08)#, decay=0.001)
# Callbacks
callbacks = get_callbacks(filepath=file_path, patience=15)
# Compile
model.compile(optimizer=myoptim, loss='binary_crossentropy', metrics=['accuracy'])
#model.summary()
model.fit_generator(train_generator,
                         steps_per_epoch=128,
                         epochs=200,
                         verbose=1,
                         validation_data=(X_valid, y_valid),
                         #validation_data = validation_generator,
                         #validation_steps = len(X_valid)/batch_size,
                         callbacks=callbacks)


Epoch 1/200
128/128 [==============================] - 805s 6s/step - loss: 0.5346 - acc: 0.7260 - val_loss: 0.6753 - val_acc: 0.5137

In [14]:
########### PRIOR TO SUBMISSION  ############
model.load_weights(filepath=file_path)
#score = gmodel.evaluate_generator(validation_generator,steps = 1)
score = model.evaluate(X_valid, y_valid)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

401/401 [==============================] - 13s 32ms/step
Test loss: 0.245187007318
Test accuracy: 0.897755611121


In [15]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
X_test_new = X_test/100+0.5

In [16]:
#predicted_test=gmodel.predict_generator(test_datagen.flow(X_test,batch_size=len(X_test)), steps=1)
#predicted_test=model.predict(X_test)
predicted_test=model.predict(X_test_new)
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
#submission.to_csv('../submit/submission11072000.csv', index=False)
#predicted_test=gmodel.predict_proba(X_test)

In [17]:
leaky_angle = [34.4721, 42.5591, 33.6352, 36.1061, 39.2340]
mask = [test['inc_angle'][i] in leaky_angle for i in range(len(test))]
column_name = 'is_iceberg'
submission.loc[mask, column_name] = 1


In [18]:
submission.to_csv('../submit/submission11132225.csv', index=False)

In [19]:
submission.head()

,id,is_iceberg
0,5941774d,0.725908
1,4023181e,0.122534
2,b20200e4,0.000241
3,e7f018bb,0.997053
4,4371c8c3,0.075933


In [20]:
s1 = pd.read_csv('../submit/submission11091153_1.csv')
s2 = pd.read_csv('../submit/submission11120000.csv')
s3 = submission.copy()
s4 = pd.read_csv('../submit/submission11131138.csv')

In [21]:
s3.is_iceberg = (s4.is_iceberg+s3.is_iceberg+s2.is_iceberg+s1.is_iceberg)/4

In [22]:
s3.to_csv('../submit/ensemble_test_3.csv',index=False)